# Optimizacion de Desarolladores por Seniority



## Variables Exogenas

### Datos

* **IA**: Intervalo entre Arribos [minutos]
* **TA**: Tiempo de Atencion [minutos]

Estan dados en base a distintas funciones de distribucion de probabilidad, obtenidas mediante _fitting_ de datos reales.


In [1]:
import dataclasses
import sys

from scipy.stats import stats

HV: int = sys.maxsize


def time_between_arrivals() -> int:
    """
    Provides the next arrival the time delta for the next arrival
    
    Returns:
        int: how many minutes until the next arrival
    """
    best_params = {'b': 8.081771083639762, 'loc': -400.05903534867156, 'scale': 400.0590353486715}
    [next_arrival_in] = stats.pareto.rvs(**best_params, size=1)
    return next_arrival_in


def service_time() -> int:
    """
    Provides the time delta for the next service
    
    Returns:
        int: how many minutes the service will take
    """
    best_params = {'a': 1.0849480846709834, 'b': 1.3832003644016413, 'c': 1.9794855101439612, 'z': 0.982926014368478,
                   'loc': 0.9477414716993133, 'scale': 1296.2750378147014}
    [busy_time] = stats.pareto.rvs(**best_params, size=1)
    return busy_time

### Control

* **NPS**: Numero de Programadores Sr
* **NPJ**: Numero de Programadores Jr

In [2]:
@dataclasses.dataclass
class ControlVector:
    """
    Control vector for the simulation
    
    Attributes:
        nps (int): Number of Senior Programmers
        npj (int): Number of Junior Programmers
    """
    nps: int
    npj: int

## Variables Endogenas

### Estado

* **NSH**: Numero de tickets en cola de prioridad alta
* **NSL**: Numero de tickets en cola de prioridad baja

In [3]:
@dataclasses.dataclass
class StatusVector:
    """
    Status vector for the simulation
    
    Attributes:
        nsh (int): Number of high priority tickets
        nsl (int): Number of low priority tickets
    """
    nsh: int
    nsl: int

### Resultados

* **PPS**: Promedio de Permanencia en el Sistema [minutos]
* **PEC**: Promedio de Permanencia en la Cola [minutos]
* **PTO**: Promedio de Tiempo Ocioso [minutos]
* **PTTS**: Porcentaje de Tickets de Baja Prioridad Atendidos por Sr [%]

In [4]:
@dataclasses.dataclass
class ResultVector:
    """
    Result vector for the simulation
    
    Attributes:
        pps (float): Average time in the system
        pec (float): Average time in the queue
        pto_jr (list[float]): Average idle time for Jrs
        pto_sr (list[float]): Average idle time for Srs
        ptts (float): Percentage of low priority tickets attended by Sr
    """
    pps: float
    pec: float
    pto_sr: list[float]
    pto_jr: list[float]
    ptts: float

## Tabla de Eventos Indepedientes

| Evento       | Evento Futuro No Condicionado | Evento Futuro Condicionado | Condicion                                 |
|--------------|-------------------------------|----------------------------|-------------------------------------------|
| LLEGADA      | LLEGADA                       | Salida Sr[i]               | NSH <= NPS \|\| (NSL > NPJ && NSH <= NPS) |
| ...          | ...                           | Salida Jr[j]               | NSL <= NPJ                                |
| Salida Sr[i] |                               | Salida Sr[i]               | NSH > NPS \|\| (NSL > NPJ && NSH > NPS)   |
| Salida Jr[j] |                               | Salida Jr[j]               | NSL > NPJ                                 |

## Tabla de Eventos Futuros

TPLL (Tiempo de Llegada del Proximo Ticket), TPSSi (Tiempo de Proxima Salida del Programador Sr i-esimo), TPSJj (Tiempo de Proximo Salida del Programador j-esimo)

In [5]:
@dataclasses.dataclass
class FutureEventsVector:
    """
    Future events vector for the simulation
    
    Attributes:
        tpll (int): Time of the next ticket arrival
        tpss (int): Time of the next Senior Programmer exit
        tpsj (int): Time of the next Junior Programmer exit
    """
    tpll: int
    tpss: list[int]
    tpsj: list[int]

    @staticmethod
    def find_min(future_events: list[int]) -> int:
        """
        Get next event in given list
        
        Args:
            future_events (list[int]): list of future events
        
        Returns:
            int: the index for the nearest event exit
        """
        return future_events.index(min(future_events))

    @staticmethod
    def find_idle(future_events: list[int]) -> int:
        """
        Get the index of the next idle programmer
        
        Args:
            future_events (list[int]): list of future events
        
        Returns:
            int: the index for the nearest idle programmer
        """
        return future_events.index(HV)

## Simulation

In [6]:
import random


def simulate(control: ControlVector, tf: int) -> ResultVector:
    """
    Simulate the system with the given control vector
    
    Args:
        control (ControlVector): the control vector for the simulation
        tf (int): the final time for the simulation
    
    Returns:
        ResultVector: the result vector for the simulation
    """
    # initial conditions
    status = StatusVector(nsh=0, nsl=0)
    future_events = FutureEventsVector(tpll=0, tpss=[HV] * control.nps, tpsj=[HV] * control.npj)
    results = ResultVector(pps=.0, pec=.0, pto_sr=[0] * control.nps, pto_jr=[0] * control.npj, ptts=.0)

    # accumulators
    stll, sts = 0, 0
    sto_sr, ito_sr = [0] * control.nps, [0] * control.nps
    sto_jr, ito_jr = [0] * control.npj, [0] * control.npj
    sta = 0
    nt = 0

    # Initialize the simulation clock
    t = 0

    # Run the simulation
    while t < tf:
        # Get the next event
        tpll = future_events.tpll
        senior_index = future_events.find_min(future_events.tpss)
        jr_index = future_events.find_min(future_events.tpsj)

        # Get the next event time
        if tpll < future_events.tpss[senior_index] and tpll < future_events.tpsj[jr_index]:
            # Event: Arrival

            # Advance the simulation clock
            t = tpll

            # Generate the next arrival
            tpll = time_between_arrivals()
            future_events.tpll = tpll
            stll += t

            # Process the arrival of a new ticket
            r = random.random()

            if r <= .41:
                # Process the arrival of a high priority ticket
                status.nsh += 1

                # Check if a Senior Programmer is available
                if status.nsh <= control.nps:
                    sr_programmer = future_events.find_idle(future_events.tpss)
                    sto_sr[sr_programmer] += t - ito_sr[sr_programmer]

                    ta = service_time()

                    # Schedule the next exit of the Senior Programmer
                    future_events.tpss[sr_programmer] = t + ta

                    sta += ta
                else:
                    # do nothing - must wait in the queue
                    ...
            else:
                # Process the arrival of a low priority ticket
                status.nsl += 1

                # Check if a Junior Programmer is available
                if status.nsl <= control.npj:
                    jr_programmer = future_events.find_idle(future_events.tpsj)
                    sto_jr[jr_programmer] += t - ito_jr[jr_programmer]

                    # Schedule the next exit of the Junior Programmer
                    ta = service_time()
                    future_events.tpsj[jr_programmer] = t + ta

                    sta += ta
                # Check if a Senior Programmer is available
                elif status.nsh <= control.nps:
                    # swap the ticket
                    status.nsl -= 1
                    status.nsh += 1

                    sr_programmer = future_events.find_idle(future_events.tpss)
                    sto_sr[sr_programmer] += t - ito_sr[sr_programmer]

                    ta = service_time()

                    # Schedule the next exit of the Senior Programmer
                    future_events.tpss[sr_programmer] = t + ta

                    sta += ta
                nt += 1
        elif future_events.tpss[senior_index] < future_events.tpsj[jr_index]:
            # Event: Senior Programmer Exit

            # Advance the simulation clock
            t = future_events.tpss[senior_index]
            sts += t

            status.nsh -= 1

            # Check if all Jrs are busy and there are more tickets to process
            if status.nsl > status.npj and status.nsh <= control.nps:
                # swap the ticket
                status.nsl -= 1
                status.nsh += 1

                # Schedule the next exit of the Senior Programmer
                ta = service_time()
                future_events.tpss[senior_index] = t + ta
                sta += ta
            # Check if there are more tickets to process
            elif status.nsh >= 1:
                ...
            else:
                # Schedule the Senior Programmer to be idle
                future_events.tpss[senior_index] = HV
                ito_sr[senior_index] = t
        else:
            # Event: Junior Programmer Exit

            # Advance the simulation clock
            t = future_events.tpsj[jr_index]
            sts += t

            status.nsl -= 1

            # Check if there are more tickets to process
            if status.nsl >= 1:
                # Schedule the next exit of the Junior Programmer
                ta = service_time()
                future_events.tpsj[jr_index] = t + ta
                sta += ta
            else:
                # Schedule the Junior Programmer to be idle
                future_events.tpsj[jr_index] = HV
                ito_jr[jr_index] = t

    # Calculate the results
    results.pps = (sts - stll) / nt
    results.pec = (sta - stll - sta) / nt
    results.pto_jr = [sto / tf for sto in sto_jr]
    results.pto_sr = [sto / tf for sto in sto_sr]
    return results